In [1]:
# ! pip install nytimesarticle
! pip install yanytapi

In [2]:
from nytimesarticle import articleAPI
from yanytapi import SearchAPI
import pandas as pd
import requests
# api = articleAPI('THFAgqFwEabG2AW7vxkxXQAEDXtZA4zK')
yourkey = ('THFAgqFwEabG2AW7vxkxXQAEDXtZA4zK')

# articles = api.search( q = 'Obama', 
#      fq = {'headline':'Obama', 'source':['Reuters','AP', 'The New York Times']}, 
#      begin_date = 20111231 )
# fq=news_desk:("Sports" "Foreign")
# articles = api.search(q = "ebola",
#       fq=news_desk:("Sports" "Foreign"), 
#      begin_date = 20030201)
#      end_date = 20030731)

In [4]:
# pull data from NYT API
# pages = [0:100]
subject = 'SARS'
body = 'World Health Organization'
section_name = 'World'
begin_date = 20030201
end_date = 20030731
# articles = requests.get(f'https://api.nytimes.com/svc/search/v2/articlesearch.json?q=SARS&fq=body:{body}ANDsection_name:{section_name}&begin_date={begin_date}&end_date={end_date}&api-key={yourkey}')
articles = requests.get(f'https://api.nytimes.com/svc/search/v2/articlesearch.json?q=SARS&fq=section_name:{section_name}&begin_date={begin_date}&end_date={end_date}&api-key={yourkey}')


In [6]:
# import pandas as pd

pd.DataFrame({
    'dates':[x['pub_date'] for x in articles.json()['response']['docs']],
    'headline':[x['headline']['main'] for x in articles.json()['response']['docs']],
    'section_name':[x['section_name'] for x in articles.json()['response']['docs']],
    'abstract':[x['abstract'] for x in articles.json()['response']['docs']],
#     'print_page':[x['print_page'] for x in articles.json()['response']['docs']]
#     'abstract':[x['keywords'] for x in articles.json()['response']['docs']]
    
})

,dates,headline,section_name,abstract
0,2003-07-31T05:00:00+0000,Toronto Turns On Rock to Drown Out SARS Fears,World,"Huge rock concert is held in Toronto, Ontario,..."
1,2003-07-06T05:00:00+0000,"SARS Declared Contained, With No Cases In Past...",World,World Heath Organization declares SARS has bee...
2,2003-07-03T05:00:00+0000,World Briefing | Americas: Canada: SARS All-Clear,World,World Health Organization removes Toronto from...
3,2003-06-26T05:00:00+0000,World Briefing | Asia: China: New Sars Case,World,China reports new case of SARS in Guangdong Pr...
4,2003-06-25T05:00:00+0000,"China Has Quelled SARS, World Health Agency Says",World,World Health Organization declares that Beijin...
5,2003-06-21T05:00:00+0000,"Isolation, an Old Medical Tool, Has SARS Fading",World,"Mystery virus SARS, which three months ago see..."
6,2003-06-18T05:00:00+0000,"As SARS Ebbs, W.H.O. Lifts Its Advisory On Taiwan",World,"WHO lifts advisory against travel to Taiwan, l..."
7,2003-06-17T05:00:00+0000,SARS Advisory Against Traveling to Taiwan Is L...,World,Beijing is now the only place that the World H...
8,2003-07-01T05:00:00+0000,Analysts See Tension in China Within the Top L...,World,China analysts say conflict between supporters...
9,2003-06-14T05:00:00+0000,W.H.O. Lifts SARS Alert in China Regions,World,World Health Organization removes travel advis...


In [7]:
# articles = api.search(q="SARS")
articles.json()['response']['docs'][0].keys()

dict_keys(['abstract', 'web_url', 'snippet', 'lead_paragraph', 'print_section', 'print_page', 'source', 'multimedia', 'headline', 'keywords', 'pub_date', 'document_type', 'news_desk', 'section_name', 'byline', 'type_of_material', '_id', 'word_count', 'uri'])

In [8]:
articles.json()['response']['docs'][7]['print_page']#[0]['name']

KeyError: 'print_page'

In [9]:
def parse_articles(articles):
    '''
    This function takes in a response to the NYT api and parses
    the articles into a list of dictionaries
    '''
    news = []
    for i in articles['response']['docs']:
        dic = {}
        dic['id'] = i['_id']
        if i['abstract'] is not None:
            dic['abstract'] = i['abstract'].encode("utf8")
        dic['headline'] = i['headline']['main'].encode("utf8")
        dic['desk'] = i['news_desk']
        dic['date'] = i['pub_date'][0:10] # cutting time of day.
        dic['section'] = i['section_name']
        if i['snippet'] is not None:
            dic['snippet'] = i['snippet'].encode("utf8")
        dic['source'] = i['source']
        dic['type'] = i['type_of_material']
        dic['url'] = i['web_url']
        dic['word_count'] = i['word_count']
        # locations
        locations = []
        for x in range(0,len(i['keywords'])):
            if 'glocations' in i['keywords'][x]['name']:
                locations.append(i['keywords'][x]['value'])
        dic['locations'] = locations
        # subject
        subjects = []
        for x in range(0,len(i['keywords'])):
            if 'subject' in i['keywords'][x]['name']:
                subjects.append(i['keywords'][x]['value'])
        dic['subjects'] = subjects   
        news.append(dic)
    return(news) 

In [85]:
parse_articles(articles)

TypeError: 'Response' object is not subscriptable

In [70]:
def get_articles(date,query):
    '''
    This function accepts a year in string format (e.g.'1980')
    and a query (e.g.'Amnesty International') and it will 
    return a list of parsed articles (in dictionaries)
    for that year.
    '''
    all_articles = []
    for i in range(0,100): #NYT limits pager to first 100 pages. But rarely will you find over 100 pages of results anyway.
        articles = api.search(q = query,
#                fq = {'source':['Reuters','AP', 'The New York Times']},
               begin_date = date + '0101',
               end_date = date + '1231',
               sort='oldest',
               page = str(i))
        articles = parse_articles(articles)
        all_articles = all_articles + articles
    return(all_articles)

In [71]:
get_articles('2003','SARS')

KeyError: 'response'

In [72]:
Amnesty_all = []
for i in range(1980,2014):
    print 'Processing' + str(i) + '...'
    Amnesty_year =  get_articles(str(i),'Amnesty International')
    Amnesty_all = Amnesty_all + Amnesty_year

SyntaxError: invalid syntax (<ipython-input-72-cb5ac759db38>, line 3)

In [73]:
api.search?

Signature: api.search(response_format=None, key=None, **kwargs)
Docstring:
Calls the API and returns a dictionary of the search results

:param response_format: the format that the API uses for its response, 
                        includes JSON (.json) and JSONP (.jsonp). 
                        Defaults to '.json'.
                        
:param key: a developer key. Defaults to key given when the articleAPI class was initialized.
File:      c:\users\travi\anaconda3\lib\site-packages\nytimesarticle.py
Type:      method


NameError: name 'ArticleAPI' is not defined

In [63]:
articleAPI

nytimesarticle.articleAPI

Object `nytimesarticle` not found.
